# 投资组合专题

In [1]:
import sys
import pandas as pd
import numpy as np

# 自行编写的包
sys.path.append('/home/ubuntu/notebooks/pycharm_projects/Investor-Sentiment')
sys.path.append('/usr/local/stata17/utilities')

# Stata
from pystata import config

# GPA
config.init('mp')


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 8-core , expiring  1 Jan 2025
Serial number: 501709301094
  Licensed to: Colin's Stata 17 MP
               Love you

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


In [2]:
df_data = pd.read_csv('/data/DataSets/investor_sentiment/FINAL_DATA_2014_2021.csv')
df_data

,Unnamed: 0,trade_date,ts_code,CON_YEAR,CON_OR,CON_NP,CON_EPS,CON_NA,CON_PB,CON_PS,...,NIPO_R,RIPO_R,SR_R,l1_TURN_R,lnPD_R,SENT_INDEX_R,SENT_INDEX_EX,SENT_INDEX_DEX,SENT_INDEX_R_EX,SENT_INDEX_R_DEX
0,7,20140102,000985.SH,2014,3.034892e+09,27421.678403,0.6857,2.017385e+09,1.3334,0.8863,...,-0.993200,-0.429099,15.745780,-0.003943,-0.208500,-1.191905,-0.571980,-0.588495,-0.435412,-0.451046
1,18,20140103,000985.SH,2014,3.033876e+09,27421.601209,0.6857,2.017584e+09,1.3213,0.8787,...,-0.993200,-0.429099,15.745780,-0.004333,-0.208500,-1.182604,-0.598652,-0.588830,-0.502679,-0.452930
2,29,20140106,000985.SH,2014,3.033345e+09,27412.012060,0.6854,2.027389e+09,1.2895,0.8619,...,-0.993200,-0.429099,15.745780,-0.003813,-0.208500,-1.175354,-0.623006,-0.589509,-0.563137,-0.455863
3,40,20140107,000985.SH,2014,3.028546e+09,27384.608096,0.6846,2.027386e+09,1.2917,0.8647,...,-0.993200,-0.429099,15.745780,-0.003643,-0.208500,-1.172984,-0.644795,-0.590505,-0.617575,-0.459753
4,51,20140108,000985.SH,2014,3.028892e+09,27355.547003,0.6838,2.028490e+09,1.2935,0.8663,...,-0.993200,-0.429099,15.745780,-0.004513,-0.208500,-1.185114,-0.664723,-0.591815,-0.666961,-0.464538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1939,21350,20211227,000985.SH,2021,6.389869e+09,53799.927579,0.7131,5.236956e+09,1.7835,1.4617,...,-0.044895,-0.374305,-2.832969,0.001832,0.097983,0.390557,1.170102,1.302731,0.960139,1.050876
1940,21361,20211228,000985.SH,2021,6.391559e+09,53788.880275,0.7129,5.237495e+09,1.7933,1.4695,...,2.955105,-0.346122,-2.832969,0.000582,0.097983,1.186737,1.127746,1.298598,0.909492,1.046500
1941,21372,20211229,000985.SH,2021,6.389955e+09,53725.674838,0.7120,5.237052e+09,1.7779,1.4571,...,-1.044895,-0.453585,-2.832969,0.000302,0.097983,0.013389,1.149856,1.293518,0.934144,1.040757
1942,21383,20211230,000985.SH,2021,6.390729e+09,53695.625032,0.7116,5.236812e+09,1.7905,1.4672,...,1.955105,-0.193822,-2.832969,0.000072,0.097983,0.972127,1.079838,1.288670,0.852272,1.035358


## MA-多空策略

In [13]:
def ma_strategy(df, factor_list, window_list):
    """
    用均值策略作为买入卖出信号
    :param df: 因子数据
    :param factor_list: 待选决策因子
    :param window_list: 待选决策窗口
    :return:
    """
    out_list = []
    # 循环因子和窗口
    for w in window_list:
        for f in factor_list:
            # 排序
            df = df.sort_values('trade_date')

            # MA均值计算
            df[f'{f}_ma_{w}'] = (df['img_neg'].rolling(w).mean())

            # 形成投资信号
            df[f'{f}_sell_signal_ma_{w}'] = df[f] >= df[f'{f}_ma_{w}']
            df[f'{f}_sell_signal_ma_{w}'] = df[f'{f}_sell_signal_ma_{w}'].shift(1)  # 向前移动一天,使用历史信息

            # 卖空指数操作
            df[f'{f}_return_ma_{w}'] = np.where(df[f'{f}_sell_signal_ma_{w}'], -1 * (df[f'{f}_sell_signal_ma_{w}'] * df['close_chg']),
                                                df['close_chg'])

            # 计算累积市值
            df[f'{f}_mv_ma_{w}'] = ((df[f'{f}_return_ma_{w}'] + 100) / 100).cumprod(axis=0)

            # 输出值列表
            out_list += [f'{f}_mv_ma_{w}', f'{f}_mdd_ma_{w}']

    # 去掉空行(MA行)
    df.dropna(axis=0, inplace=True)

    # 对比基准
    df['mv_shareindex'] = ((df['close_chg'] + 100) / 100).cumprod(axis=0)

    return df[['trade_date', 'ts_code', 'mv_shareindex', ] + out_list]


ma_strategy(df_data, ['img_neg', 'tex_neg', 'SENT_INDEX', 'SENT_INDEX_R'], [5, 10, 15, 20])


,trade_date,ts_code,mv_shareindex,img_neg_mv_ma_5,img_neg_mdd_ma_5,tex_neg_mv_ma_5,tex_neg_mdd_ma_5,SENT_INDEX_mv_ma_5,SENT_INDEX_mdd_ma_5,SENT_INDEX_R_mv_ma_5,...,SENT_INDEX_R_mv_ma_15,SENT_INDEX_R_mdd_ma_15,img_neg_mv_ma_20,img_neg_mdd_ma_20,tex_neg_mv_ma_20,tex_neg_mdd_ma_20,SENT_INDEX_mv_ma_20,SENT_INDEX_mdd_ma_20,SENT_INDEX_R_mv_ma_20,SENT_INDEX_R_mdd_ma_20
114,20140624,000985.SH,1.005968,0.97348,-3.472673,1.035729,-3.472673,1.00771,-3.472673,1.00771,...,1.00771,-3.472673,1.00771,-3.472673,1.00771,-3.472673,1.00771,-3.472673,1.00771,-3.472673
115,20140625,000985.SH,1.000909,0.978376,-3.472673,1.040939,-3.472673,1.002641,-3.472673,1.002641,...,1.002641,-3.472673,1.012778,-3.472673,1.012778,-3.472673,1.002641,-3.472673,1.002641,-3.472673
116,20140626,000985.SH,1.011837,0.967694,-3.472673,1.052304,-3.472673,1.013589,-3.472673,1.013589,...,1.013589,-3.472673,1.00172,-3.472673,1.023836,-3.472673,1.013589,-3.472673,1.013589,-3.472673
117,20140627,000985.SH,1.015974,0.963738,-3.472673,1.056607,-3.472673,1.017733,-3.472673,1.009445,...,1.009445,-3.472673,0.997625,-3.472673,1.028022,-3.472673,1.009445,-3.472673,1.009445,-3.472673
118,20140630,000985.SH,1.024543,0.971867,-3.472673,1.047694,-3.472673,1.026317,-3.472673,1.017959,...,1.017959,-3.472673,1.00604,-3.472673,1.019351,-3.472673,1.017959,-3.472673,1.017959,-3.472673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1939,20211227,000985.SH,2.101280,4.492784,-8.187696,0.607613,-8.187696,1.265764,-8.187696,2.143179,...,1.922442,-8.187696,2.618814,-8.187696,0.471174,-8.187696,1.5538,-8.187696,1.964329,-8.187696
1940,20211228,000985.SH,2.116337,4.46059,-8.187696,0.611967,-8.187696,1.256693,-8.187696,2.127822,...,1.908667,-8.187696,2.600048,-8.187696,0.47455,-8.187696,1.542666,-8.187696,1.950253,-8.187696
1941,20211229,000985.SH,2.097057,4.419954,-8.187696,0.617542,-8.187696,1.268142,-8.187696,2.147206,...,1.926055,-8.187696,2.576361,-8.187696,0.478874,-8.187696,1.556719,-8.187696,1.96802,-8.187696
1942,20211230,000985.SH,2.114861,4.38243,-8.187696,0.612299,-8.187696,1.257376,-8.187696,2.165435,...,1.942406,-8.187696,2.554489,-8.187696,0.474808,-8.187696,1.543503,-8.187696,1.984728,-8.187696


In [4]:
def ma_strategy(df, factor_list, window_list):
    """
    用均值策略作为买入卖出信号
    :param df: 因子数据
    :param factor_list: 待选决策因子
    :param window_list: 待选决策窗口
    :return:
    """
    out_list = []
    # 循环因子和窗口
    for w in window_list:
        for f in factor_list:
            # 排序
            df.sort_values('trade_date', inplace=True)
            # MA均值计算
            df[f'{f}_ma_{w}'] = (df['img_neg'].rolling(w).mean())

            # 形成投资信号
            df[f'{f}_sell_signal_ma_{w}'] = df[f] >= df[f'{f}_ma_{w}']
            df[f'{f}_sell_signal_ma_{w}'] = df[f'{f}_sell_signal_ma_{w}'].shift(1)  # 向前移动一天,使用历史信息

            # 卖空指数操作
            df[f'{f}_return_ma_{w}'] = np.where(df[f'{f}_sell_signal_ma_{w}'], -1 * (df[f'{f}_sell_signal_ma_{w}'] * df['close_chg']),
                                                df['close_chg'])

            # 计算累积市值
            df[f'{f}_mv_ma_{w}'] = ((df[f'{f}_return_ma_{w}'] + 100) / 100).cumprod(axis=0)

            # 计算最大回撤
            df[f'{f}_mdd_ma_{w}'] = df[f'{f}_return_ma_{w}'].cummin(axis=0)

            # 计算夏普比率
            df[f'{f}_sharp_ma_{w}'] = (df[f'{f}_return_ma_{w}'].mean()) / df[f'{f}_return_ma_{w}'].std(ddof=0) * np.sqrt(250)

            # 卡玛比率
            df[f'{f}_kama_ma_{w}'] = (df[f'{f}_return_ma_{w}'].mean() * 250) / (df[f'{f}_mdd_ma_{w}'].min())

            # 输出值列表
            out_list += [f'{f}_mv_ma_{w}', f'{f}_mdd_ma_{w}', f'{f}_sharp_ma_{w}', f'{f}_kama_ma_{w}']

    # 去掉空行(MA行)
    df.dropna(axis=0, inplace=True)

    # 对比基准
    df['mv_shareindex'] = ((df['close_chg'] + 100) / 100).cumprod(axis=0)
    df['mdd_shareindex'] = df['close_chg'].cummin(axis=0)
    df['sharp_shareindex'] = (df['close_chg'].mean()) / df['close_chg'].std(ddof=0) * np.sqrt(250)

    return df[['trade_date', 'ts_code', 'mv_shareindex', 'mdd_shareindex', 'sharp_shareindex'] + out_list]


#ma策略
ma_strategy(df_data, ['img_neg', 'tex_neg', 'SENT_INDEX', 'SENT_INDEX_R'], [5, 10, 15, 20])


,trade_date,ts_code,mv_shareindex,mdd_shareindex,sharp_shareindex,img_neg_mv_ma_5,img_neg_mdd_ma_5,img_neg_sharp_ma_5,img_neg_kama_ma_5,tex_neg_mv_ma_5,...,tex_neg_sharp_ma_20,tex_neg_kama_ma_20,SENT_INDEX_mv_ma_20,SENT_INDEX_mdd_ma_20,SENT_INDEX_sharp_ma_20,SENT_INDEX_kama_ma_20,SENT_INDEX_R_mv_ma_20,SENT_INDEX_R_mdd_ma_20,SENT_INDEX_R_sharp_ma_20,SENT_INDEX_R_kama_ma_20
38,20140304,000985.SH,0.998806,-0.119353,0.515628,1.060409,-1.390642,0.91447,-2.648137,1.05796,...,-0.333287,0.966535,1.011265,-3.206304,0.338528,-0.981727,1.011265,-3.206304,0.476251,-1.380813
39,20140305,000985.SH,0.993038,-0.577564,0.515628,1.054285,-1.390642,0.91447,-2.648137,1.05185,...,-0.333287,0.966535,1.005424,-3.206304,0.338528,-0.981727,1.005424,-3.206304,0.476251,-1.380813
40,20140306,000985.SH,0.995065,-0.577564,0.515628,1.056437,-1.390642,0.91447,-2.648137,1.049703,...,-0.333287,0.966535,1.007476,-3.206304,0.338528,-0.981727,1.007476,-3.206304,0.476251,-1.380813
41,20140307,000985.SH,0.992266,-0.577564,0.515628,1.059408,-1.390642,0.91447,-2.648137,1.04675,...,-0.333287,0.966535,1.004643,-3.206304,0.338528,-0.981727,1.004643,-3.206304,0.476251,-1.380813
42,20140310,000985.SH,0.957808,-3.472673,0.515628,1.022618,-3.472673,0.91447,-2.648137,1.0104,...,-0.333287,0.966535,0.969755,-3.472673,0.338528,-0.981727,0.969755,-3.472673,0.476251,-1.380813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1939,20211227,000985.SH,2.022546,-8.187696,0.515628,4.405193,-8.187696,0.91447,-2.648137,0.571006,...,-0.333287,0.966535,1.511618,-8.187696,0.338528,-0.981727,1.911002,-8.187696,0.476251,-1.380813
1940,20211228,000985.SH,2.037039,-8.187696,0.515628,4.373627,-8.187696,0.91447,-2.648137,0.575098,...,-0.333287,0.966535,1.500786,-8.187696,0.338528,-0.981727,1.897308,-8.187696,0.476251,-1.380813
1941,20211229,000985.SH,2.018482,-8.187696,0.515628,4.333783,-8.187696,0.91447,-2.648137,0.580337,...,-0.333287,0.966535,1.514458,-8.187696,0.338528,-0.981727,1.914592,-8.187696,0.476251,-1.380813
1942,20211230,000985.SH,2.035618,-8.187696,0.515628,4.29699,-8.187696,0.91447,-2.648137,0.57541,...,-0.333287,0.966535,1.501601,-8.187696,0.338528,-0.981727,1.930847,-8.187696,0.476251,-1.380813
